# Recommending Books using LightGCN

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


In [3]:
import torch

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

from PIL import Image
import requests

from torch_geometric.data import HeteroData
from torch_geometric.loader import NeighborSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from datetime import datetime

## Preprocessing the dataset

### Import necessary libraries

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
from torch import nn, optim, Tensor
import torch.sparse as sparse

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn import LGConv

In [5]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.version.cuda)         # Should print the CUDA version PyTorch is built with


True
11.8


### Read in the raw data

In [6]:
users_df = pd.read_csv('data/BX-Users.csv', sep=';', encoding='latin-1')
# rename 'Location' and 'Age' to lowercase
users_df.rename(columns={'Location': 'location', 'Age': 'age'}, inplace=True)
# Example age bins
bins = [0, 18, 35, 55, 75, float('inf')]
labels = ['0-18', '19-35', '36-55', '56-75', '76+']

users_df['age'].fillna(users_df['age'].median(), inplace=True)

users_df['age_group'] = pd.cut(users_df['age'], bins=bins, labels=labels, right=False)

# rename 'User-ID' to 'user_id' 
users_df.rename(columns={'User-ID': 'user_id'}, inplace=True)

items_df = pd.read_csv('data/BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')
# rename 'Book-Title' to 'title', 'Book-Author' to 'author'
items_df.rename(columns={'Book-Title': 'title', 'Book-Author': 'author', 'Publisher':'publisher'}, inplace=True)
# rename 'ISBN' to 'item_id'
items_df.rename(columns={'ISBN': 'item_id'}, inplace=True)



C:\Users\aimer\AppData\Local\Temp\ipykernel_8960\2270659622.py:15: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  items_df = pd.read_csv('data/BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')


In [7]:
items_df.head()

,item_id,title,author,Year-Of-Publication,publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [8]:
users_df.head()

,user_id,location,age,age_group
0,1,"nyc, new york, usa",32.0,19-35
1,2,"stockton, california, usa",18.0,19-35
2,3,"moscow, yukon territory, russia",32.0,19-35
3,4,"porto, v.n.gaia, portugal",17.0,0-18
4,5,"farnborough, hants, united kingdom",32.0,19-35


In [9]:
items_df.head()

,item_id,title,author,Year-Of-Publication,publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [10]:
items_df.shape

(271360, 8)

In [11]:
items_df['author'] = items_df['author'].fillna('NA')
items_df['publisher'] = items_df['publisher'].fillna('NA')

In [12]:
items_df.isna().sum()

item_id                0
title                  0
author                 0
Year-Of-Publication    0
publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [13]:
# delete rows with missing values
# items_df.dropna(inplace=True)

In [14]:
items_df.shape

(271360, 8)

In [15]:
ratings_df = pd.read_csv('data/BX-Book-Ratings.csv', sep=';', encoding='latin-1')  
# rename 'User-ID' to 'user_id', 'Book-Rating' to 'rating', 'ISBN' to 'item_id'
ratings_df.rename(columns={'User-ID': 'user_id', 'Book-Rating': 'rating', 'ISBN': 'item_id'}, inplace=True)
ratings_df.head()


,user_id,item_id,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [16]:
ratings_df.shape

(1149780, 3)

In [17]:

# Preprocessing
ratings_df = ratings_df.loc[ratings_df['item_id'].isin(items_df['item_id'].unique()) & ratings_df['user_id'].isin(users_df['user_id'].unique())]

print("full rating dataframe size: ", str(ratings_df.shape))
# # Keep the 100k highest ratings
ratings_df = ratings_df[ratings_df['rating'] >= 8] # .iloc[:100000]
print("subset rating dataframe size: ", str(ratings_df.shape))

# subsert users_df where user_id is in ratings_df
users_df = users_df[users_df['user_id'].isin(ratings_df['user_id'].unique())].copy()
# subsert items_df where item_id is in ratings_df
items_df = items_df[items_df['item_id'].isin(ratings_df['item_id'].unique())].copy()

print(users_df.shape)
print(items_df.shape)

full rating dataframe size:  (1031136, 3)
subset rating dataframe size:  (223807, 3)
(47074, 4)
(98417, 8)


In [18]:
# Create mappings
user_mapping = {userid: i for i, userid in enumerate(users_df['user_id'].unique())}
item_mapping = {isbn: i for i, isbn in enumerate(items_df['item_id'].unique())}

In [19]:
user_mapping[16]

1

In [20]:
import pickle

# Save user_mapping
with open('models/user_mapping.pkl', 'wb') as f:
    pickle.dump(user_mapping, f)

# Save item_mapping
with open('models/item_mapping.pkl', 'wb') as f:
    pickle.dump(item_mapping, f)


In [21]:
# Count users and items
num_users = len(user_mapping)
num_items = len(item_mapping)
num_total = num_users + num_items

# Map user and item indices
users_df['user_idx'] = users_df['user_id'].map(user_mapping)
items_df['item_idx'] = items_df['item_id'].map(item_mapping)
ratings_df['user_idx'] = ratings_df['user_id'].map(user_mapping)
ratings_df['item_idx'] = ratings_df['item_id'].map(item_mapping)

# Merge the user and item features with the ratings
# merged_df = ratings_df.merge(users_df, on='user_id').merge(items_df, on='item_id')

users_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47074 entries, 11 to 278853
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   user_id    47074 non-null  int64   
 1   location   47074 non-null  object  
 2   age        47074 non-null  float64 
 3   age_group  47074 non-null  category
 4   user_idx   47074 non-null  int64   
dtypes: category(1), float64(1), int64(2), object(1)
memory usage: 1.8+ MB


In [22]:
ratings_df.item_idx.min()

0

In [36]:
items_df.columns

Index(['item_id', 'title', 'author', 'Year-Of-Publication', 'publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'item_idx', 'subject_idx',
       'author_idx'],
      dtype='object')

In [23]:
items_df.head()

,item_id,title,author,Year-Of-Publication,publisher,Image-URL-S,Image-URL-M,Image-URL-L,item_idx
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,0
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,1
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,2
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,3
6,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,http://images.amazon.com/images/P/0425176428.0...,4


### Preprocessing the dataset: transforming the user features and item features

In [24]:
# # fill missing values of locations, age_group, subject, author with 'unknown'
# def fillna_category(df, column):
#     df[column] = df[column].astype('category')
#     df[column].cat.add_categories('unknown')
#     df[column].fillna('unknown', inplace=True)

# check if there are any missing values
items_df.isna().sum()

# delete rows with missing values in 'title' and 'author'
# items_df.dropna(subset=['title', 'author'], inplace=True)


item_id                0
title                  0
author                 0
Year-Of-Publication    0
publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
item_idx               0
dtype: int64

In [25]:

# Function to map categories to indices
def map_categories(df, column):
    unique_values = df[column].unique()
    value_to_idx = {value: idx for idx, value in enumerate(unique_values)}
    return df[column].map(value_to_idx), len(unique_values)

# Map each categorical feature
users_df['location_idx'], num_locations = map_categories(users_df, 'location')
users_df['age_group_idx'], num_age_groups = map_categories(users_df, 'age_group')
items_df['subject_idx'], num_subjects = map_categories(items_df, 'publisher')
items_df['author_idx'], num_authors = map_categories(items_df, 'author')

### Prepare Input data for the Neural Network Model

In [26]:
# Convert indices to tensors
user_location_indices = torch.tensor(users_df['location_idx'].values, dtype=torch.long)
user_age_group_indices = torch.tensor(users_df['age_group_idx'].values, dtype=torch.long)
item_subject_indices = torch.tensor(items_df['subject_idx'].values, dtype=torch.long)
item_author_indices = torch.tensor(items_df['author_idx'].values, dtype=torch.long)


In [27]:
# Initialize HeteroData
data = HeteroData()

# Assigning integer-encoded features
data['user'].x = torch.cat([user_location_indices.unsqueeze(1), user_age_group_indices.unsqueeze(1)], dim=1)
data['item'].x = torch.cat([item_subject_indices.unsqueeze(1), item_author_indices.unsqueeze(1)], dim=1)

# Creating edge index from ratings dataframe
edge_index = torch.tensor(ratings_df[['user_idx', 'item_idx']].values.T, dtype=torch.long)
edge_features = torch.tensor(ratings_df['rating'].values, dtype=torch.float)
# Create edge labels based on the rating
edge_labels = (edge_features >= 8).long()  # This will create a tensor of 0s and 1s

# Assign edge features (ratings), edge type = 'Rating Edge'
data['user', 'Rating Edge', 'item'].edge_index = edge_index
data['user', 'Rating Edge', 'item'].edge_attr = edge_features
# Assign edge labels to the HeteroData object
data['user', 'Rating Edge', 'item'].edge_label = edge_labels
# Assign edge label indices (in this case, they are the same as edge indices)
data['user', 'Rating Edge', 'item'].edge_label_index = edge_index


In [28]:
edge_labels

tensor([1, 1, 1,  ..., 1, 1, 1])

In [29]:
data

HeteroData(
  user={ x=[47074, 2] },
  item={ x=[98417, 2] },
  (user, Rating Edge, item)={
    edge_index=[2, 223807],
    edge_attr=[223807],
    edge_label=[223807],
    edge_label_index=[2, 223807],
  }
)

In [30]:
import pandas as pd

# Assuming ratings_df, users_df, and items_df are your DataFrames

# 1. Check for Invalid or Missing Values in ratings_df
if ratings_df.isnull().values.any():
    print("There are missing values in ratings_df.")
else:
    print("No missing values in ratings_df.")

# 2. Verify User and Item Index Alignment
max_user_idx = ratings_df['user_idx'].max()
max_item_idx = ratings_df['item_idx'].max()

if max_user_idx >= len(users_df) or max_item_idx >= len(items_df):
    print("User or Item indices in ratings_df exceed the number of rows in users_df or items_df.")
else:
    print("User and Item indices in ratings_df are correctly aligned.")

# 3. Check for Duplicate Rows
if ratings_df.duplicated(subset=['user_idx', 'item_idx']).any():
    print("There are duplicate user-item pairs in ratings_df.")
else:
    print("No duplicate user-item pairs in ratings_df.")

# 4. Inspect Index Continuity
user_idx_continuous = set(ratings_df['user_idx']) == set(range(max_user_idx + 1))
item_idx_continuous = set(ratings_df['item_idx']) == set(range(max_item_idx + 1))

if not user_idx_continuous or not item_idx_continuous:
    print("User or Item indices in ratings_df are not continuous.")
else:
    print("User and Item indices in ratings_df are continuous.")


No missing values in ratings_df.
User and Item indices in ratings_df are correctly aligned.
No duplicate user-item pairs in ratings_df.
User and Item indices in ratings_df are continuous.


In [31]:
# Assuming edge_labels is a tensor of edge labels

# Number of edges in ratings_df
num_edges_in_df = ratings_df.shape[0]

# Number of labels in edge_labels
num_labels = edge_labels.shape[0]

# Check if every edge has a corresponding label
if num_edges_in_df == num_labels:
    print("Every edge has a corresponding label.")
else:
    print(f"Mismatch: {num_edges_in_df} edges in DataFrame, but {num_labels} labels.")


Every edge has a corresponding label.


In [32]:
# Check if indices start from 0 and are continuous

# Check for user indices
min_user_idx = ratings_df['user_idx'].min()
max_user_idx = ratings_df['user_idx'].max()
user_indices_continuous = set(ratings_df['user_idx']) == set(range(min_user_idx, max_user_idx + 1))

# Check for item indices
min_item_idx = ratings_df['item_idx'].min()
max_item_idx = ratings_df['item_idx'].max()
item_indices_continuous = set(ratings_df['item_idx']) == set(range(min_item_idx, max_item_idx + 1))

# Print the results
if min_user_idx != 0 or not user_indices_continuous:
    print("User indices are either not starting from 0 or not continuous.")
else:
    print("User indices start from 0 and are continuous.")

if min_item_idx != 0 or not item_indices_continuous:
    print("Item indices are either not starting from 0 or not continuous.")
else:
    print("Item indices start from 0 and are continuous.")


User indices start from 0 and are continuous.
Item indices start from 0 and are continuous.


## Data split

In [33]:
def split_edges(edge_index, edge_attr, proportions=(0.8, 0.1, 0.1)):
    """
    Splits the edges of a graph into training, validation, and test sets.

    This function randomly shuffles the edges of a graph and then splits them into
    three subsets according to the specified proportions. It is primarily used in
    graph learning tasks where you need separate edge sets for training, validation,
    and testing the model.

    Parameters:
    edge_index (Tensor): A 2xN tensor where N is the number of edges, and each column
                         represents an edge in the format [source, target].
    edge_attr (Tensor): A tensor containing attributes for each edge in edge_index.
                        The length of this tensor should match the number of edges in edge_index.
    proportions (tuple, optional): A tuple representing the proportions for splitting the edges
                                    into training, validation, and test sets. Defaults to (0.8, 0.1, 0.1).

    Returns:
    tuple: A tuple containing three tuples, each for the training, validation, and test sets.
           Each inner tuple contains a subset of edge_index and the corresponding edge_attr.
           Format: ((train_edge_index, train_edge_attr),
                    (val_edge_index, val_edge_attr),
                    (test_edge_index, test_edge_attr))
    """
    num_edges = edge_index.size(1)
    all_indices = torch.randperm(num_edges)

    train_size = int(proportions[0] * num_edges)
    val_size = int(proportions[1] * num_edges)

    train_indices = all_indices[:train_size]
    val_indices = all_indices[train_size:train_size + val_size]
    test_indices = all_indices[train_size + val_size:]

    return (edge_index[:, train_indices], edge_attr[train_indices]), \
           (edge_index[:, val_indices], edge_attr[val_indices]), \
           (edge_index[:, test_indices], edge_attr[test_indices])


In [34]:
def create_data_subset(source_data, edge_index, edge_attr, edge_label_index, edge_label):
    """
    Creates a subset of graph data from a source heterogeneous graph.

    This function constructs a new HeteroData object (a heterogeneous graph representation) 
    by selecting specific edges and their attributes from the source data. It's typically 
    used in graph neural network tasks to create subsets of data for training, validation, 
    or testing. The function assumes the graph has a bipartite structure with 'user' and 
    'item' node types and edges representing 'Rating Edge'.

    Parameters:
    source_data (HeteroData): The source heterogeneous graph data from which the subset is created.
                              This should be an instance of HeteroData with 'user' and 'item' node types.
    edge_index (Tensor): A 2xN tensor specifying the edges in the subset. Each column is an edge
                         in the format [source, target], typically representing a 'user'-'item' interaction.
    edge_attr (Tensor): A tensor containing attributes for each edge specified in edge_index.
    edge_label_index (Tensor): A 2xM tensor specifying the edges for which labels are provided.
                               This is typically a subset or the same as edge_index.
    edge_label (Tensor): A tensor containing labels for each edge specified in edge_label_index.

    Returns:
    HeteroData: A new HeteroData object representing the subset of the original graph. This subset
                includes node features for 'user' and 'item' types and the specified edges with their
                attributes and labels.
    """
    subset = HeteroData()
    subset['user'].x = source_data['user'].x
    subset['item'].x = source_data['item'].x
    subset['user', 'Rating Edge', 'item'].edge_index = edge_index
    subset['user', 'Rating Edge', 'item'].edge_attr = edge_attr
    subset['user', 'Rating Edge', 'item'].edge_label_index = edge_label_index
    subset['user', 'Rating Edge', 'item'].edge_label = edge_label
    return subset


In [35]:
def split_edges(edge_index, edge_attr, edge_label_index, edge_label, proportions=(0.8, 0.1, 0.1)):
    """
    Splits the edges of a graph into training, validation, and test sets.

    This function shuffles and divides the edges of a graph into three subsets based on
    the specified proportions. It is useful for preparing graph data for machine learning tasks,
    ensuring distinct edge sets for training, validation, and testing phases.

    Parameters:
    edge_index (Tensor): A 2xN tensor with each column representing an edge as [source, target].
    edge_attr (Tensor): Attributes associated with each edge in edge_index.
    edge_label_index (Tensor): A 2xN tensor specifying the edges for which labels are provided.
    edge_label (Tensor): Labels for each edge specified in edge_label_index.
    proportions (tuple of float, optional): The proportions to split the edges into training,
                                            validation, and test sets. Defaults to (0.8, 0.1, 0.1).

    Returns:
    tuple: Three tuples corresponding to the training, validation, and test sets. Each tuple contains
           the edge_index, edge_attr, edge_label_index, and edge_label for that subset.
    """
    num_edges = edge_index.size(1)
    all_indices = torch.randperm(num_edges)

    train_size = int(proportions[0] * num_edges)
    val_size = int(proportions[1] * num_edges)

    train_indices = all_indices[:train_size]
    val_indices = all_indices[train_size:train_size + val_size]
    test_indices = all_indices[train_size + val_size:]

    return ((edge_index[:, train_indices], edge_attr[train_indices],
             edge_label_index[:, train_indices], edge_label[train_indices]),
            (edge_index[:, val_indices], edge_attr[val_indices],
             edge_label_index[:, val_indices], edge_label[val_indices]),
            (edge_index[:, test_indices], edge_attr[test_indices],
             edge_label_index[:, test_indices], edge_label[test_indices]))

# Apply the split
train_split, val_split, test_split = split_edges(
    data['user', 'Rating Edge', 'item'].edge_index, 
    data['user', 'Rating Edge', 'item'].edge_attr,
    data['user', 'Rating Edge', 'item'].edge_label_index,
    data['user', 'Rating Edge', 'item'].edge_label
)

# Create subsets
train_data = create_data_subset(data, *train_split)
val_data = create_data_subset(data, *val_split)
test_data = create_data_subset(data, *test_split)


In [36]:
train_data

HeteroData(
  user={ x=[47074, 2] },
  item={ x=[98417, 2] },
  (user, Rating Edge, item)={
    edge_index=[2, 179045],
    edge_attr=[179045],
    edge_label_index=[2, 179045],
    edge_label=[179045],
  }
)

In [37]:
val_data

HeteroData(
  user={ x=[47074, 2] },
  item={ x=[98417, 2] },
  (user, Rating Edge, item)={
    edge_index=[2, 22380],
    edge_attr=[22380],
    edge_label_index=[2, 22380],
    edge_label=[22380],
  }
)

### Graph Dataloader

* The critical purpose of graph dataloader is to create mini-batches so that the model training won't take too much resources to run.

In [38]:
from torch_geometric.loader import LinkNeighborLoader
# Assuming `data` is your HeteroData object
# Assuming edge_label_index and edge_labels are already defined in data

# Configure the LinkLoader
train_edge_label_index = train_data['user', 'Rating Edge', 'item'].edge_label_index
train_edge_label = train_data['user', 'Rating Edge', 'item'].edge_label
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=1.0,  # Ratio of negative samples to positive samples    
    edge_label_index=(('user', 'Rating Edge', 'item'), train_edge_label_index),
    edge_label=train_edge_label,
    batch_size=128,  # Define your batch size
    shuffle=True  # Shuffle the data for training
)

# Configure the LinkLoader
val_edge_label_index = val_data['user', 'Rating Edge', 'item'].edge_label_index
val_edge_label = val_data['user', 'Rating Edge', 'item'].edge_label
val_loader = LinkNeighborLoader(
    data=val_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=1.0,  # Ratio of negative samples to positive samples    
    edge_label_index=(('user', 'Rating Edge', 'item'), val_edge_label_index),
    edge_label=val_edge_label,
    batch_size=128,  # Define your batch size
    shuffle=True  # Shuffle the data for training
)

# Configure the LinkLoader
test_edge_label_index = test_data['user', 'Rating Edge', 'item'].edge_label_index
test_edge_label = test_data['user', 'Rating Edge', 'item'].edge_label
test_loader = LinkNeighborLoader(
    data=test_data,
    num_neighbors=[20, 10],
    neg_sampling_ratio=1.0,  # Ratio of negative samples to positive samples    
    edge_label_index=(('user', 'Rating Edge', 'item'), test_edge_label_index),
    edge_label=test_edge_label,
    batch_size=128,  # Define your batch size
    shuffle=True  # Shuffle the data for training
)


In [81]:
train_data

HeteroData(
  user={ x=[47074, 2] },
  item={ x=[98417, 2] },
  (user, Rating Edge, item)={
    edge_index=[2, 179045],
    edge_attr=[179045],
    edge_label_index=[2, 179045],
    edge_label=[179045],
  }
)

## Implementing the LightGCN architecture

* specify the model architectures which include node features and edge features.

In [32]:
import torch
import torch.nn as nn
from torch.sparse import mm as sparse_mm

class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, num_layers,
                 num_locations, num_age_groups, num_subjects, num_authors):
        super(LightGCN, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers

        # Initialize user and item embeddings
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)

        # Initialize additional feature embeddings
        self.location_embedding = nn.Embedding(num_locations, embedding_dim)
        self.age_group_embedding = nn.Embedding(num_age_groups, embedding_dim)
        self.subject_embedding = nn.Embedding(num_subjects, embedding_dim)
        self.author_embedding = nn.Embedding(num_authors, embedding_dim)

        # Initialization
        nn.init.normal_(self.user_embedding.weight, std=0.1)
        nn.init.normal_(self.item_embedding.weight, std=0.1)
        nn.init.normal_(self.location_embedding.weight, std=0.1)
        nn.init.normal_(self.age_group_embedding.weight, std=0.1)
        nn.init.normal_(self.subject_embedding.weight, std=0.1)
        nn.init.normal_(self.author_embedding.weight, std=0.1)

    def forward(self, user_indices, item_indices, location_indices, age_group_indices, subject_indices, author_indices, adj_matrix):
        # Initial user and item embeddings
        user_emb = self.user_embedding(user_indices)
        item_emb = self.item_embedding(item_indices)

        # Embeddings for additional features
        location_emb = self.location_embedding(location_indices)
        age_group_emb = self.age_group_embedding(age_group_indices)
        subject_emb = self.subject_embedding(subject_indices)
        author_emb = self.author_embedding(author_indices)

        # Combine base embeddings with feature embeddings
        user_emb = user_emb + location_emb + age_group_emb
        item_emb = item_emb + subject_emb + author_emb

        # Debug: Print dimensions for troubleshooting
        print("Adjacency Matrix Shape:", adj_matrix.shape)
        print("User Embeddings Shape:", user_emb.shape)
        print("Item Embeddings Shape:", item_emb.shape)

        all_embeddings = torch.cat([user_emb, item_emb], dim=0)

        print("All Embeddings Shape:", all_embeddings.shape)

        # List to hold all embeddings for each layer
        all_user_embs = [user_emb]
        all_item_embs = [item_emb]

        # Perform graph convolutions
        for _ in range(self.num_layers):
            # Ensure dimensions are compatible for matrix multiplication
            if adj_matrix.shape[0] != all_embeddings.shape[0]:
                raise ValueError("Dimension mismatch: adj_matrix and all_embeddings must have the same number of rows")

            all_embeddings = sparse_mm(adj_matrix, all_embeddings)

            user_emb = all_embeddings[:self.num_users]
            item_emb = all_embeddings[self.num_users:]

            all_user_embs.append(user_emb)
            all_item_embs.append(item_emb)

        # Compute final embeddings as the mean of all layers' embeddings
        final_user_emb = torch.mean(torch.stack(all_user_embs), dim=0)
        final_item_emb = torch.mean(torch.stack(all_item_embs), dim=0)

        # Predict ratings by computing the dot product of user and item embeddings
        scores = torch.sum(final_user_emb * final_item_emb, dim=1)

        self.cached_embeddings = all_embeddings  # Cache the embeddings

        return scores
    
    def get_user_embeddings(self, user_indices):
        user_emb = self.user_embedding(user_indices)
        # You can add additional features if necessary
        return user_emb

    def get_item_embeddings(self, item_indices):
        item_emb = self.item_embedding(item_indices)
        # You can add additional features if necessary
        return item_emb

    

# Example usage:
# model = LightGCN(num_users=1000, num_items=500, embedding_dim=64, num_layers=3,
#                  num_locations=10, num_age_groups=5, num_subjects=20, num_authors=50)
# ... (setup data and training loop) ...


In [71]:
def bpr_loss(users_emb, pos_items_emb, neg_items_emb):
    """
    Bayesian Personalized Ranking (BPR) loss using log sigmoid.
    
    Args:
        users_emb (Tensor): Embeddings of users.
        pos_items_emb (Tensor): Embeddings of positive items.
        neg_items_emb (Tensor): Embeddings of negative items.

    Returns:
        Tensor: BPR loss.
    """
    pos_scores = (users_emb * pos_items_emb).sum(dim=1)
    neg_scores = (users_emb * neg_items_emb).sum(dim=1)
    scores = pos_scores - neg_scores

    # Debugging: Check the range of scores
    print("Max score:", scores.max().item(), "Min score:", scores.min().item())

    loss = -torch.mean(torch.nn.functional.logsigmoid(scores))
    return loss

* When computing the logarithm of the sigmoid function, there's a risk of numerical instability, especially when the argument of the sigmoid function (pos_scores - neg_scores) takes large positive or negative values. In these cases, the sigmoid function can return values very close to 0 or 1, leading to issues when taking the logarithm (since log(0) is undefined and can lead to nan values).
* 
To enhance the stability, you can use the torch.nn.functional.logsigmoid function, which is a numerically stable version of the logarithm of the sigmoid function.

## LightGCN Model Training

In [72]:
# Number of unique locations
num_locations = data['user'].x[:, 0].unique().size(0)
# Number of unique age groups
num_age_groups = data['user'].x[:, 1].unique().size(0)

# Number of unique subjects
num_subjects = data['item'].x[:, 0].unique().size(0)
# Number of unique authors
num_authors = data['item'].x[:, 1].unique().size(0)
# Number of unique users
num_users = data['user', 'Rating Edge', 'item'].edge_index[0].unique().size(0)
# Number of unique items
num_items = data['user', 'Rating Edge', 'item'].edge_index[1].unique().size(0)

print("Number of locations:", num_locations)
print("Number of age groups:", num_age_groups)
print("Number of subjects:", num_subjects)
print("Number of authors:", num_authors)
print("Number of users:", num_users)
print("Number of items:", num_items)


Number of locations: 14239
Number of age groups: 5
Number of subjects: 8751
Number of authors: 42696
Number of users: 47074
Number of items: 98417


### Initialize the LightGCN Model

In [73]:
# Instantiate the LightGCN model
model = LightGCN(num_users=num_users, 
                 num_items=num_items, 
                 embedding_dim=64, 
                 num_layers=3,
                 num_locations=num_locations, 
                 num_age_groups=num_age_groups, 
                 num_subjects=num_subjects, 
                 num_authors=num_authors)


### Define the Optimizer and Loss Function

In [86]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

# Define your loss function (assuming BPR loss)
def bpr_loss(users_emb, pos_items_emb, neg_items_emb):
    """
    Bayesian Personalized Ranking (BPR) loss using log sigmoid.
    
    Args:
        users_emb (Tensor): Embeddings of users.
        pos_items_emb (Tensor): Embeddings of positive items.
        neg_items_emb (Tensor): Embeddings of negative items.

    Returns:
        Tensor: BPR loss.
    """
    pos_scores = (users_emb * pos_items_emb).sum(dim=1)
    neg_scores = (users_emb * neg_items_emb).sum(dim=1)
    scores = pos_scores - neg_scores

    # Debugging: Check the range of scores
    # print("Max score:", scores.max().item(), "Min score:", scores.min().item())

    loss = -torch.mean(torch.nn.functional.logsigmoid(scores))
    return loss

In [87]:
import torch
import torch.optim as optim

# Assuming the LightGCN model is defined as LightGCN
model = LightGCN(num_users=num_users, 
                 num_items=num_items, 
                 embedding_dim=64,  # Example embedding dimension
                 num_layers=3,      # Example number of layers
                 num_locations=num_locations, 
                 num_age_groups=num_age_groups, 
                 num_subjects=num_subjects, 
                 num_authors=num_authors)

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

def train(model, train_loader, optimizer):
    model.train()  # Set the model to training mode
    total_loss = 0.0

    for batch in train_loader:
        optimizer.zero_grad()

        edge_label_index = batch['user', 'Rating Edge', 'item'].edge_label_index
        edge_labels = batch['user', 'Rating Edge', 'item'].edge_label

        user_indices = edge_label_index[0]
        item_indices = edge_label_index[1]

        pos_item_indices = item_indices[edge_labels == 1]
        neg_item_indices = item_indices[edge_labels == 0]

        # Limit the number of samples to the smallest size
        min_size = min(user_indices.size(0), pos_item_indices.size(0), neg_item_indices.size(0))
        user_indices = user_indices[:min_size]
        pos_item_indices = pos_item_indices[:min_size]
        neg_item_indices = neg_item_indices[:min_size]

        # Get embeddings
        users_emb = model.get_user_embeddings(user_indices)
        pos_items_emb = model.get_item_embeddings(pos_item_indices)
        neg_items_emb = model.get_item_embeddings(neg_item_indices)

        # Compute BPR loss
        loss = bpr_loss(users_emb, pos_items_emb, neg_items_emb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)


def validate(model, val_loader):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0

    with torch.no_grad():
        for batch in val_loader:
            edge_label_index = batch['user', 'Rating Edge', 'item'].edge_label_index
            edge_labels = batch['user', 'Rating Edge', 'item'].edge_label

            user_indices = edge_label_index[0]
            item_indices = edge_label_index[1]

            pos_item_indices = item_indices[edge_labels == 1]
            neg_item_indices = item_indices[edge_labels == 0]

            # Limit the number of samples to the smallest size
            min_size = min(user_indices.size(0), pos_item_indices.size(0), neg_item_indices.size(0))
            user_indices = user_indices[:min_size]
            pos_item_indices = pos_item_indices[:min_size]
            neg_item_indices = neg_item_indices[:min_size]

            users_emb = model.get_user_embeddings(user_indices)
            pos_items_emb = model.get_item_embeddings(pos_item_indices)
            neg_items_emb = model.get_item_embeddings(neg_item_indices)

            loss = bpr_loss(users_emb, pos_items_emb, neg_items_emb)
            total_loss += loss.item()

    return total_loss / len(val_loader)


def test(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0

    with torch.no_grad():
        for batch in test_loader:
            edge_label_index = batch['user', 'Rating Edge', 'item'].edge_label_index
            edge_labels = batch['user', 'Rating Edge', 'item'].edge_label

            user_indices = edge_label_index[0]
            item_indices = edge_label_index[1]

            pos_item_indices = item_indices[edge_labels == 1]
            neg_item_indices = item_indices[edge_labels == 0]

            # Limit the number of samples to the smallest size
            min_size = min(user_indices.size(0), pos_item_indices.size(0), neg_item_indices.size(0))
            user_indices = user_indices[:min_size]
            pos_item_indices = pos_item_indices[:min_size]
            neg_item_indices = neg_item_indices[:min_size]

            users_emb = model.get_user_embeddings(user_indices)
            pos_items_emb = model.get_item_embeddings(pos_item_indices)
            neg_items_emb = model.get_item_embeddings(neg_item_indices)

            loss = bpr_loss(users_emb, pos_items_emb, neg_items_emb)
            total_loss += loss.item()

    return total_loss / len(test_loader)


# Assuming model, train_loader, val_loader, and optimizer are already defined
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer)
    val_loss = validate(model, val_loader)
    test_loss = test(model, test_loader)

    print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Test Loss: {test_loss:.4f}")


Epoch 1/10 - Training Loss: 0.6369, Validation Loss: 0.5548, Test Loss: 0.5572
Epoch 2/10 - Training Loss: 0.5415, Validation Loss: 0.5290, Test Loss: 0.5263
Epoch 3/10 - Training Loss: 0.5202, Validation Loss: 0.5191, Test Loss: 0.5162
Epoch 4/10 - Training Loss: 0.5152, Validation Loss: 0.5129, Test Loss: 0.5100
Epoch 5/10 - Training Loss: 0.5122, Validation Loss: 0.5097, Test Loss: 0.5118
Epoch 6/10 - Training Loss: 0.5115, Validation Loss: 0.5128, Test Loss: 0.5126
Epoch 7/10 - Training Loss: 0.5121, Validation Loss: 0.5096, Test Loss: 0.5082
Epoch 8/10 - Training Loss: 0.5099, Validation Loss: 0.5121, Test Loss: 0.5102
Epoch 9/10 - Training Loss: 0.5096, Validation Loss: 0.5120, Test Loss: 0.5135
Epoch 10/10 - Training Loss: 0.5095, Validation Loss: 0.5039, Test Loss: 0.5095


In [88]:
# save the model
torch.save(model.state_dict(), 'models/lightgcn_model_with_features.pt')

In [30]:
# Number of unique locations
num_locations = data['user'].x[:, 0].unique().size(0)
# Number of unique age groups
num_age_groups = data['user'].x[:, 1].unique().size(0)

# Number of unique subjects
num_subjects = data['item'].x[:, 0].unique().size(0)
# Number of unique authors
num_authors = data['item'].x[:, 1].unique().size(0)
# Number of unique users
num_users = data['user', 'Rating Edge', 'item'].edge_index[0].unique().size(0)
# Number of unique items
num_items = data['user', 'Rating Edge', 'item'].edge_index[1].unique().size(0)

print("Number of locations:", num_locations)
print("Number of age groups:", num_age_groups)
print("Number of subjects:", num_subjects)
print("Number of authors:", num_authors)
print("Number of users:", num_users)
print("Number of items:", num_items)


Number of locations: 14239
Number of age groups: 5
Number of subjects: 8751
Number of authors: 42696
Number of users: 47074
Number of items: 98417


In [33]:
# Assuming the LightGCN class and other parameters (like num_users, num_items, etc.) are defined
model = LightGCN(num_users=num_users, 
                 num_items=num_items, 
                 embedding_dim=64, 
                 num_layers=3, 
                 num_locations=num_locations, 
                 num_age_groups=num_age_groups, 
                 num_subjects=num_subjects, 
                 num_authors=num_authors)

model.load_state_dict(torch.load('models/lightgcn_model_with_features.pt'))
model.eval()


LightGCN(
  (user_embedding): Embedding(47074, 64)
  (item_embedding): Embedding(98417, 64)
  (location_embedding): Embedding(14239, 64)
  (age_group_embedding): Embedding(5, 64)
  (subject_embedding): Embedding(8751, 64)
  (author_embedding): Embedding(42696, 64)
)

In [34]:
def score_all_items_batched(model, num_users, num_items, batch_size=1000):
    all_scores = torch.zeros(num_users, num_items)

    # Iterate over users in batches
    for user_batch_start in range(0, num_users, batch_size):
        user_batch_end = min(user_batch_start + batch_size, num_users)
        user_indices = torch.arange(user_batch_start, user_batch_end)
        user_embeddings = model.get_user_embeddings(user_indices)

        # Iterate over items in batches
        for item_batch_start in range(0, num_items, batch_size):
            item_batch_end = min(item_batch_start + batch_size, num_items)
            item_indices = torch.arange(item_batch_start, item_batch_end)
            item_embeddings = model.get_item_embeddings(item_indices)

            # Compute scores for the current batch
            batch_scores = torch.mm(user_embeddings, item_embeddings.t())
            all_scores[user_batch_start:user_batch_end, item_batch_start:item_batch_end] = batch_scores

    return all_scores


def exclude_interacted_items(scores, interaction_history):
    for user_id in range(scores.size(0)):
        interacted_items = interaction_history.get(user_id, [])
        scores[user_id, interacted_items] = -float('inf')
    return scores

def get_top_k_recommendations(scores, k=5):
    top_k_items = torch.topk(scores, k, dim=1).indices
    return top_k_items

def get_interaction_history(data):
    interaction_history = {}

    edge_index = data['user', 'Rating Edge', 'item'].edge_index

    # Iterate over all interactions (both positive and negative)
    for index in range(edge_index.size(1)):  # Iterate over the number of edges
        user_id = edge_index[0, index].item()
        item_id = edge_index[1, index].item()

        if user_id not in interaction_history:
            interaction_history[user_id] = []
        interaction_history[user_id].append(item_id)

    return interaction_history


In [35]:

interaction_history = get_interaction_history(data)

# Load the model
model.load_state_dict(torch.load('models/lightgcn_model_with_features.pt'))
model.eval()

# Score all items for each user
scores = score_all_items_batched(model, num_users, num_items,  batch_size=500)

# Exclude already interacted items
scores = exclude_interacted_items(scores, interaction_history)

# Get top-5 recommendations
top_5_recommendations = get_top_k_recommendations(scores, k=5)

# Display the recommendations
for user_id in range(10):  # Modify as needed for your specific use case
    print(f"User {user_id}: {top_5_recommendations[user_id]}")

# save top_5_recommendations to a pands dataframe and a csv file
top_5_recommendations_df = pd.DataFrame(top_5_recommendations)
top_5_recommendations_df.to_csv('data/top_5_recommendations.csv', index=False)


User 0: tensor([0, 2, 3, 1, 8])
User 1: tensor([0, 3, 2, 1, 5])
User 2: tensor([0, 3, 2, 6, 1])
User 3: tensor([0, 3, 2, 1, 5])
User 4: tensor([0, 3, 2, 1, 5])
User 5: tensor([0, 3, 2, 1, 5])
User 6: tensor([0, 3, 2, 5, 1])
User 7: tensor([0, 3, 2, 1, 5])
User 8: tensor([0, 2, 3, 1, 5])
User 9: tensor([0, 3, 2, 1, 4])


In [41]:
top_5_recommendations_df.columns = ['item1', 'item2', 'item3', 'item4', 'item5']
# turn index into user_idx column in top_5_recommendations_df
top_5_recommendations_df['user_idx'] = top_5_recommendations_df.index
top_5_recommendations_df.head()

,item1,item2,item3,item4,item5,user_idx
0,0,2,3,1,8,0
1,0,3,2,1,5,1
2,0,3,2,6,1,2
3,0,3,2,1,5,3
4,0,3,2,1,5,4


In [45]:
# merge top_5_recommendations_df with users_df, drop location_idx and age_group_idx

top_5_recommendations_users = pd.merge(top_5_recommendations_df, users_df.drop(columns=['location_idx', 'age_group_idx']), on='user_idx', how='left')
top_5_recommendations_users.head()

,item1,item2,item3,item4,item5,user_idx,user_id,location,age,age_group
0,0,2,3,1,8,0,12,"fort bragg, california, usa",32.0,19-35
1,0,3,2,1,5,1,16,"albuquerque, new mexico, usa",32.0,19-35
2,0,3,2,6,1,2,26,"bellevue, washington, usa",32.0,19-35
3,0,3,2,1,5,3,32,"portland, oregon, usa",32.0,19-35
4,0,3,2,1,5,4,39,"cary, north carolina, usa",32.0,19-35
